In [86]:
!pip install folium

In [87]:
import numpy as np # library to handle data in a vectorized manner



from pprint import pprint

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
from urllib.request import urlopen

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Importing libraries:

Getting the URL:

In [88]:
wiki_URL="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(wiki_URL)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


Converting it to a JSON and then to a dataframe:

In [89]:
table_class="wikitable sortable"  #from inspecting element you get the class wikitable sortable jquery-tablesorter
response=requests.get(wiki_URL)
print(response.status_code)

200


Parse data from the HTML : Next we create a BeautifulSoup object and using the find() method extract the relevant information,which in our case is the \<table> tag. There can be many tables in a single Wikipedia page, so to specify the table we also pass the “class” or the “id” attribute of the \<table>

In [90]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
index=soup.find('table',{'class':table_class})
print(soup)
print(index)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"36bf7290-1dd3-4f34-9720-ad50b756be16","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","

 We read the HTML table into a list of dataframe object using read_html(). This returns a list. Next we convert the list into a DataFrame.

In [91]:
df=pd.read_html(str(index))
# convert list to dataframe
df=pd.DataFrame(df[0],index=None)
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [92]:
rows, colums=df.shape

list1=[]
for x in range(0,rows,1):
    print(x,df.loc[x])
    if df.loc[x].get('Borough')!="Not assigned" and df.loc[x].get('Neighbourhood')=="Not assigned":
        df.loc[x]['Neighbourhood']=df.loc[x]['Borough']
        list1.append({'Postal Code':df.loc[x].get('Postal Code'),'Borough':df.loc[x].get('Borough'),'Neighbourhood':df.loc[x].get('Neighbourhood')})
    elif df.loc[x].get('Borough')!="Not assigned" and df.loc[x].get('Neighbourhood')!="Not assigned":
        list1.append({'Postal Code':df.loc[x].get('Postal Code'),'Borough':df.loc[x].get('Borough'),'Neighbourhood':df.loc[x].get('Neighbourhood')})
    else:
        continue

print(*list1)

0 Postal Code               M1A
Borough          Not assigned
Neighbourhood    Not assigned
Name: 0, dtype: object
1 Postal Code               M2A
Borough          Not assigned
Neighbourhood    Not assigned
Name: 1, dtype: object
2 Postal Code             M3A
Borough          North York
Neighbourhood     Parkwoods
Name: 2, dtype: object
3 Postal Code                   M4A
Borough                North York
Neighbourhood    Victoria Village
Name: 3, dtype: object
4 Postal Code                            M5A
Borough                   Downtown Toronto
Neighbourhood    Regent Park, Harbourfront
Name: 4, dtype: object
5 Postal Code                                   M6A
Borough                                North York
Neighbourhood    Lawrence Manor, Lawrence Heights
Name: 5, dtype: object
6 Postal Code                                              M7A
Borough                                     Downtown Toronto
Neighbourhood    Queen's Park, Ontario Provincial Government
Name: 6, dtype: obje

In [93]:
df2=pd.DataFrame(list1) 
print(df.shape)
df2

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [94]:
"I check how many Borough were not assigned and I noticed that they were not Borough assigned with a non assign Neighbourhood"""
df3=df[df['Neighbourhood'].map(lambda x: str(x)=="Not assigned")]
print(df3.shape)
df3.head()

(77, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned


In [95]:
"""This another way to remove the Not assigned Borough"""
df4=df[df['Borough'].map(lambda x: str(x)!="Not assigned")]
print(df4.shape)
df4.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Finally, after checking my results. I have decided to work with df2. Then, I sorted the data using index.

In [96]:
df2.sort_index(axis=0)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Trying to obtain coordinates of Toronto using geolocators:

In [97]:
address = 'TORONTO, CANADA'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a map of Toronto 

In [98]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

Get the latitude and longitude of each borough and neighbourhood

In [99]:
rows, colums=df2.shape
print(rows)

103


In [100]:
address=[]

for x in range(0,rows,1):
    borough, neighbourhood = (df2.loc[x].get('Borough'),df2.loc[x].get('Neighbourhood'))
    if ("," in neighbourhood):              #I decided to divide the Borough that contains two neighbourhood
        address_list=neighbourhood.split(",")
        address.append([borough,address_list[0]])
        address.append([borough,address_list[1]])
    else:
        address.append([borough,neighbourhood])

print(*address)

['North York', 'Parkwoods'] ['North York', 'Victoria Village'] ['Downtown Toronto', 'Regent Park'] ['Downtown Toronto', ' Harbourfront'] ['North York', 'Lawrence Manor'] ['North York', ' Lawrence Heights'] ['Downtown Toronto', "Queen's Park"] ['Downtown Toronto', ' Ontario Provincial Government'] ['Etobicoke', 'Islington Avenue'] ['Etobicoke', ' Humber Valley Village'] ['Scarborough', 'Malvern'] ['Scarborough', ' Rouge'] ['North York', 'Don Mills'] ['East York', 'Parkview Hill'] ['East York', ' Woodbine Gardens'] ['Downtown Toronto', 'Garden District'] ['Downtown Toronto', ' Ryerson'] ['North York', 'Glencairn'] ['Etobicoke', 'West Deane Park'] ['Etobicoke', ' Princess Gardens'] ['Scarborough', 'Rouge Hill'] ['Scarborough', ' Port Union'] ['North York', 'Don Mills'] ['East York', 'Woodbine Heights'] ['Downtown Toronto', 'St. James Town'] ['York', 'Humewood-Cedarvale'] ['Etobicoke', 'Eringate'] ['Etobicoke', ' Bloordale Gardens'] ['Scarborough', 'Guildwood'] ['Scarborough', ' Morningsid

In [101]:
for x in range(0,len(address),1):
    print(','.join(address[x]))

North York,Parkwoods
North York,Victoria Village
Downtown Toronto,Regent Park
Downtown Toronto, Harbourfront
North York,Lawrence Manor
North York, Lawrence Heights
Downtown Toronto,Queen's Park
Downtown Toronto, Ontario Provincial Government
Etobicoke,Islington Avenue
Etobicoke, Humber Valley Village
Scarborough,Malvern
Scarborough, Rouge
North York,Don Mills
East York,Parkview Hill
East York, Woodbine Gardens
Downtown Toronto,Garden District
Downtown Toronto, Ryerson
North York,Glencairn
Etobicoke,West Deane Park
Etobicoke, Princess Gardens
Scarborough,Rouge Hill
Scarborough, Port Union
North York,Don Mills
East York,Woodbine Heights
Downtown Toronto,St. James Town
York,Humewood-Cedarvale
Etobicoke,Eringate
Etobicoke, Bloordale Gardens
Scarborough,Guildwood
Scarborough, Morningside
East Toronto,The Beaches
Downtown Toronto,Berczy Park
York,Caledonia-Fairbanks
Scarborough,Woburn
East York,Leaside
Downtown Toronto,Central Bay Street
Downtown Toronto,Christie
Scarborough,Cedarbrae
North 

In [102]:
latitude_list=[]
longitude_list=[]

# define the dataframe columns
column_names = ['Borough', 'Neighbourhood','Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for x in range(0,len(address),1):
    str_address=",".join(address[x])
    location = geolocator.geocode(str_address)
    if location!=None:  # With this if I remove None types. In other words, neighbourhood without coordinates.
        latitude = location.latitude
        longitude = location.longitude
        neighborhoods = neighborhoods.append({'Borough': address[x][0],
                                              'Neighbourhood': address[x][1],
                                              'Latitude': latitude,
                                              'Longitude': longitude}, ignore_index=True)

In [103]:
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.761124,-79.324059
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Regent Park,43.655513,-79.358182
3,North York,Lawrence Manor,43.722079,-79.437507
4,North York,Lawrence Heights,43.722778,-79.450933


#####  However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [104]:
neighborhoods_data = neighborhoods [neighborhoods['Borough'].map(lambda x: "Toronto" in x)].reset_index(drop=True)
neighborhoods_data

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Regent Park,43.655513,-79.358182
1,Downtown Toronto,Queen's Park,43.663217,-79.386290
2,Downtown Toronto,Garden District,43.653182,-79.375107
3,Downtown Toronto,Ryerson,43.658260,-79.378030
4,Downtown Toronto,St. James Town,43.670650,-79.375941
5,East Toronto,The Beaches,43.671024,-79.296712
6,Downtown Toronto,Central Bay Street,43.661425,-79.386067
7,Downtown Toronto,Christie,43.663287,-79.419014
8,Downtown Toronto,Richmond,43.653516,-79.368524
9,Downtown Toronto,Adelaide,43.650298,-79.380477


## Let's see our results plotted.

In [105]:
# add markers to map
for lat, lng, borough, neighbourhood in zip(neighborhoods_data['Latitude'], neighborhoods_data['Longitude'], neighborhoods_data['Borough'], neighborhoods_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Replicating New York analysis in Toronto. Thus, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [106]:
CLIENT_ID = 'CB245Q52IIDUCQ1VXR2HBN5HANNGGSNTWW2CZ0K2Z4PQWAZT' # your Foursquare ID
CLIENT_SECRET = 'M4O5SZNTHG2JBVJSERHF10H2NC5IX3OMC20IIKHDLA2KHFWB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CB245Q52IIDUCQ1VXR2HBN5HANNGGSNTWW2CZ0K2Z4PQWAZT
CLIENT_SECRET:M4O5SZNTHG2JBVJSERHF10H2NC5IX3OMC20IIKHDLA2KHFWB


###### Let's explore the first neighborhood in our dataframe.

In [107]:
neighborhoods_data.loc[0, 'Neighbourhood']

'Regent Park'

Get the Regent Park latitude and longitude values.

In [108]:
neighborhood_latitude = neighborhoods_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park are 43.6555134, -79.358182.


### Now, let's get the top 100 venues that are in Regent Park within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [109]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CB245Q52IIDUCQ1VXR2HBN5HANNGGSNTWW2CZ0K2Z4PQWAZT&client_secret=M4O5SZNTHG2JBVJSERHF10H2NC5IX3OMC20IIKHDLA2KHFWB&v=20180605&ll=43.6555134,-79.358182&radius=500&limit=100'

Send the GET request and examine the results

In [110]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f94e54605724978506a5dac'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 43.6600134045, 'lng': -79.35197387056236},
   'sw': {'lat': 43.651013395499994, 'lng': -79.36439012943764}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5612b1cc498e3dd742af0dc8',
       'name': 'Impact Kitchen',
       'location': {'address': '573 King St E',
        'crossStreet': 'at St Lawrence St',
        'lat': 43.65636850543279,
        'lng': -79.35697968750694,
        'labeledLatLngs': [{'label': '

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [111]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [112]:
pprint(results)

{'meta': {'code': 200, 'requestId': '5f94e54605724978506a5dac'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
                                                 'items': [{'reasonName': 'globalInteractionReason',
                                                            'summary': 'This '
                                                                       'spot '
                                                                       'is '
                                                                       'popular',
                                                            'type': 'general'}]},
                                     'referralId': 'e-0-5612b1cc498e3dd742af0dc8-0',
                                     'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
                                                                        'suffix': '.png'},
                                                            

In [113]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Impact Kitchen,Restaurant,43.656369,-79.356980
1,Corktown Common,Park,43.655618,-79.356211
2,Underpass Park,Park,43.655764,-79.354806
3,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
4,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
5,Sumach Espresso,Coffee Shop,43.658135,-79.359515
6,Tandem Coffee,Coffee Shop,43.653559,-79.361809
7,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
8,Roselle Desserts,Bakery,43.653447,-79.362017
9,Body Blitz Spa East,Spa,43.654735,-79.359874


And how many venues were returned by Foursquare?

In [114]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

29 venues were returned by Foursquare.


# Explore Neighborhoods in Toronto

### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [115]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [116]:
# type your answer here
toronto_venues = getNearbyVenues(names=neighborhoods_data['Neighbourhood'],
                                   latitudes=neighborhoods_data['Latitude'],
                                   longitudes=neighborhoods_data['Longitude']
                                  )


Regent Park
Queen's Park
Garden District
 Ryerson
St. James Town
The Beaches
Central Bay Street
Christie
Richmond
 Adelaide
Dufferin
 Dovercourt Village
 Union Station
Little Portugal
 Trinity
The Danforth West
 Riverdale
 Design Exchange
Brockton
 Parkdale Village
 The Beaches West
 Victoria Hotel
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
 Forest Hill Road Park
High Park
 The Junction South
 Lawrence Park
The Annex
 North Midtown
Parkdale
 Roncesvalles
Davisville
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
Summerhill West
 Rathnelly
 King and Spadina
Rosedale
St. James Town
Church and Wellesley


### Let's check the size of the resulting dataframe

In [117]:
print(toronto_venues.shape)
toronto_venues.head()

(2244, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.655513,-79.358182,Impact Kitchen,43.656369,-79.356980,Restaurant
1,Regent Park,43.655513,-79.358182,Corktown Common,43.655618,-79.356211,Park
2,Regent Park,43.655513,-79.358182,Underpass Park,43.655764,-79.354806,Park
3,Regent Park,43.655513,-79.358182,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
4,Regent Park,43.655513,-79.358182,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


Let's check how many venues were returned for each neighborhood

In [118]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Chinatown,78,78,78,78,78,78
Design Exchange,100,100,100,100,100,100
Dovercourt Village,9,9,9,9,9,9
Forest Hill Road Park,10,10,10,10,10,10
Harbord,70,70,70,70,70,70
King and Spadina,78,78,78,78,78,78
Lawrence Park,51,51,51,51,51,51
North Midtown,74,74,74,74,74,74


### Let's find out how many unique categories can be curated from all the returned venues

In [119]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 254 uniques categories.


# 3. Analyze Each Neighborhood

In [120]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent 

And let's examine the new dataframe size.

In [121]:
toronto_onehot.shape

(2244, 254)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [122]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.03,0.0,0.0,0.0,0.0,0.010000,0.000000,0.010000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.000000,0.020000,0.0,0.000000,0.01,0.00,0.0,0.0,0.0,0.060000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.030000,0.000000,0.080000,0.0,0.00,0.0,0.0,0.0,0.000000,0.010000,0.0,0.0,

### Let's confirm the new size

In [123]:
toronto_grouped.shape

(47, 254)

### Let's print each neighborhood along with the top 5 most common venues

In [124]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2         Café  0.06
3          Gym  0.05
4        Hotel  0.05


---- Chinatown----
            venue  freq
0     Coffee Shop  0.10
1  Clothing Store  0.08
2           Hotel  0.04
3           Plaza  0.03
4           Diner  0.03


---- Design Exchange----
            venue  freq
0     Coffee Shop  0.12
1  Clothing Store  0.07
2            Café  0.04
3  Sandwich Place  0.03
4  Cosmetics Shop  0.03


---- Dovercourt Village----
                  venue  freq
0                  Park  0.22
1                  Café  0.22
2  Brazilian Restaurant  0.11
3                   Bar  0.11
4           Pizza Place  0.11


---- Forest Hill Road Park----
                venue  freq
0  Light Rail Station   0.3
1                Park   0.2
2        Skating Rink   0.2
3        Liquor Store   0.1
4         Coffee Shop   0.1


---- Harbord----
                           venue  freq
0                           Café  0.10
1  Vegetaria

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [125]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [126]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Gym,Hotel,American Restaurant,Japanese Restaurant,Asian Restaurant,Gastropub,Clothing Store
1,Chinatown,Coffee Shop,Clothing Store,Hotel,Electronics Store,Theater,Plaza,Middle Eastern Restaurant,Sushi Restaurant,Diner,Cosmetics Shop
2,Design Exchange,Coffee Shop,Clothing Store,Café,Sandwich Place,Middle Eastern Restaurant,Cosmetics Shop,Bookstore,Diner,Theater,Japanese Restaurant
3,Dovercourt Village,Park,Café,Coffee Shop,Bar,Restaurant,Pizza Place,Brazilian Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,Forest Hill Road Park,Light Rail Station,Skating Rink,Park,Supermarket,Coffee Shop,Liquor Store,Eastern European Restaurant,Electronics Store,Escape Room,Women's Store


# 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [127]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
pprint(kmeans.labels_[0:10])

array([4, 4, 4, 0, 3, 0, 4, 0, 4, 0], dtype=int32)


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [128]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [129]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Adelaide,Coffee Shop,Restaurant,Café,Gym,Hotel,American Restaurant,Japanese Restaurant,Asian Restaurant,Gastropub,Clothing Store
1,4,Chinatown,Coffee Shop,Clothing Store,Hotel,Electronics Store,Theater,Plaza,Middle Eastern Restaurant,Sushi Restaurant,Diner,Cosmetics Shop
2,4,Design Exchange,Coffee Shop,Clothing Store,Café,Sandwich Place,Middle Eastern Restaurant,Cosmetics Shop,Bookstore,Diner,Theater,Japanese Restaurant
3,0,Dovercourt Village,Park,Café,Coffee Shop,Bar,Restaurant,Pizza Place,Brazilian Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,3,Forest Hill Road Park,Light Rail Station,Skating Rink,Park,Supermarket,Coffee Shop,Liquor Store,Eastern European Restaurant,Electronics Store,Escape Room,Women's Store


In [130]:
toronto_merged = neighborhoods_data
toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Regent Park,43.655513,-79.358182
1,Downtown Toronto,Queen's Park,43.663217,-79.386290
2,Downtown Toronto,Garden District,43.653182,-79.375107
3,Downtown Toronto,Ryerson,43.658260,-79.378030
4,Downtown Toronto,St. James Town,43.670650,-79.375941


### I need to rename neighbourhood to neighborhood

In [131]:
toronto_merged=toronto_merged.rename(columns={'Neighbourhood': 'Neighborhood'})
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Regent Park,43.655513,-79.358182
1,Downtown Toronto,Queen's Park,43.663217,-79.386290
2,Downtown Toronto,Garden District,43.653182,-79.375107
3,Downtown Toronto,Ryerson,43.658260,-79.378030
4,Downtown Toronto,St. James Town,43.670650,-79.375941


In [132]:
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Regent Park,43.655513,-79.358182,4,Coffee Shop,Park,Pub,Pharmacy,Athletics & Sports,Bus Stop,Breakfast Spot,Mediterranean Restaurant,Café,Spa
1,Downtown Toronto,Queen's Park,43.663217,-79.386290,4,Coffee Shop,Gay Bar,Japanese Restaurant,Park,Yoga Studio,Dance Studio,Bubble Tea Shop,Sculpture Garden,Salon / Barbershop,Sake Bar
2,Downtown Toronto,Garden District,43.653182,-79.375107,4,Gastropub,Italian Restaurant,Clothing Store,Cosmetics Shop,Coffee Shop,Hotel,Lingerie Store,Restaurant,Japanese Restaurant,Café
3,Downtown Toronto,Ryerson,43.658260,-79.378030,4,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Sandwich Place,Diner,Spa,Burger Joint,Ramen Restaurant,Japanese Restaurant
4,Downtown Toronto,St. James Town,43.670650,-79.375941,4,Coffee Shop,Grocery Store,Sandwich Place,Pharmacy,Juice Bar,Café,Men's Store,Breakfast Spot,Food & Drink Shop,Caribbean Restaurant


Finally, let's visualize the resulting clusters

In [133]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you

### Cluster 1

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,The Beaches,Beach,Japanese Restaurant,Park,Bar,Breakfast Spot,Thai Restaurant,Sandwich Place,Juice Bar,Skating Rink,Burger Joint
10,Dufferin,Bar,Bakery,Café,Coffee Shop,Sandwich Place,Vietnamese Restaurant,Cocktail Bar,Mexican Restaurant,Restaurant,Beer Store
11,Dovercourt Village,Park,Café,Coffee Shop,Bar,Restaurant,Pizza Place,Brazilian Restaurant,Farmers Market,Falafel Restaurant,Event Space
13,Little Portugal,Bar,Café,Coffee Shop,Korean Restaurant,Restaurant,Cocktail Bar,Bakery,Portuguese Restaurant,Dive Bar,Jazz Club
16,Riverdale,Vietnamese Restaurant,Bakery,Chinese Restaurant,Bar,Fast Food Restaurant,Grocery Store,Baseball Field,Light Rail Station,Asian Restaurant,Café
18,Brockton,Bar,Park,Vietnamese Restaurant,Bakery,Korean Restaurant,Auto Workshop,Dive Bar,Coffee Shop,Jazz Club,Sake Bar
19,Parkdale Village,Tibetan Restaurant,Bar,Diner,Restaurant,Pharmacy,Indian Restaurant,Bakery,Bank,Japanese Restaurant,Donut Shop
20,The Beaches West,Beach,Japanese Restaurant,Park,Bar,Breakfast Spot,Thai Restaurant,Sandwich Place,Juice Bar,Skating Rink,Burger Joint
22,Lawrence Park,Sushi Restaurant,Italian Restaurant,Coffee Shop,Bakery,Bank,Pharmacy,Pub,Asian Restaurant,Fast Food Restaurant,Sandwich Place
24,Davisville North,Sushi Restaurant,Italian Restaurant,Coffee Shop,Park,Indian Restaurant,Pub,Convenience Store,Café,Sandwich Place,Rental Car Location


### Cluster 2

In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,High Park,Convenience Store,Pool,Pet Store,Gym,Pub,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Farmers Market,Falafel Restaurant


### Cluster 3

In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Moore Park,Gym,Playground,Trail,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Women's Store


### Cluster 4

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Forest Hill Road Park,Light Rail Station,Skating Rink,Park,Supermarket,Coffee Shop,Liquor Store,Eastern European Restaurant,Electronics Store,Escape Room,Women's Store
37,Swansea,Park,Pilates Studio,Dance Studio,Women's Store,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Falafel Restaurant


### Cluster 5

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park,Coffee Shop,Park,Pub,Pharmacy,Athletics & Sports,Bus Stop,Breakfast Spot,Mediterranean Restaurant,Café,Spa
1,Queen's Park,Coffee Shop,Gay Bar,Japanese Restaurant,Park,Yoga Studio,Dance Studio,Bubble Tea Shop,Sculpture Garden,Salon / Barbershop,Sake Bar
2,Garden District,Gastropub,Italian Restaurant,Clothing Store,Cosmetics Shop,Coffee Shop,Hotel,Lingerie Store,Restaurant,Japanese Restaurant,Café
3,Ryerson,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Sandwich Place,Diner,Spa,Burger Joint,Ramen Restaurant,Japanese Restaurant
4,St. James Town,Coffee Shop,Grocery Store,Sandwich Place,Pharmacy,Juice Bar,Café,Men's Store,Breakfast Spot,Food & Drink Shop,Caribbean Restaurant
6,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Yoga Studio,Beer Bar,Spa,Smoothie Shop,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant
7,Christie,Korean Restaurant,Grocery Store,Coffee Shop,Café,Karaoke Bar,Ethiopian Restaurant,Mexican Restaurant,Indian Restaurant,Ice Cream Shop,Park
8,Richmond,Coffee Shop,Theater,Café,Restaurant,Italian Restaurant,Yoga Studio,German Restaurant,Breakfast Spot,Middle Eastern Restaurant,Event Space
9,Adelaide,Coffee Shop,Restaurant,Café,Gym,Hotel,American Restaurant,Japanese Restaurant,Asian Restaurant,Gastropub,Clothing Store
12,Union Station,Clothing Store,Hotel,Gym,Restaurant,Café,Coffee Shop,Gastropub,American Restaurant,Cosmetics Shop,Asian Restaurant
